# YOLO Model Comparison Experiments

## Automated Testing Across All YOLOv8 Models

This notebook automatically runs parking detection experiments across all YOLOv8 model variants:
- **yolov8n.pt** (Nano) - Fastest, least accurate
- **yolov8s.pt** (Small) - Balanced speed/accuracy
- **yolov8m.pt** (Medium) - Good accuracy
- **yolov8l.pt** (Large) - High accuracy
- **yolov8x.pt** (Extra Large) - Highest accuracy, slowest

## Features:
- Automatic model downloading if missing
- Comprehensive performance comparison
- Tracking issue analysis
- Consolidated results export

In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
import json
import os
import time
import pandas as pd
from datetime import datetime
from collections import defaultdict
import matplotlib.pyplot as plt

In [2]:
class ParkingDetector:
    def __init__(self, model_path, video_path):
        self.model_path = model_path
        self.video_path = video_path
        self.model_name = os.path.splitext(os.path.basename(model_path))[0]
        self.video_name = os.path.splitext(os.path.basename(video_path))[0]
        
        # Find parking areas file
        self.areas_file = f'../output/parking-areas-{self.video_name}.json'
        
        # Results
        self.results = {
            'experiment_id': f"{self.model_name}_{self.video_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
            'model_path': model_path,
            'video_path': video_path,
            'areas_file': self.areas_file,
            'fps_data': [],
            'parked_vehicles': [],
            'tracking_issues': [],  # New: track flickering/lost detections
            'detection_stats': {},
            'performance': {}
        }
        
        print(f"Model: {self.model_name}")
        print(f"Video: {self.video_name}")
        print(f"Looking for areas: {self.areas_file}")
        
    def load_model_and_areas(self):
        # Load YOLO model
        print("\nLoading model...")
        self.model = YOLO(self.model_path)
        print(f"Model loaded: {self.model_name}")
        
        # Load parking areas
        if not os.path.exists(self.areas_file):
            print(f"Parking areas not found: {self.areas_file}")
            print("   Run define-park-areas.ipynb first!")
            return False
            
        with open(self.areas_file, 'r') as f:
            areas_data = json.load(f)
            
        self.parking_areas = [np.array(area) for area in areas_data['parking_areas']]
        print(f"Loaded {len(self.parking_areas)} parking areas")
        return True
        
    def is_in_parking_area(self, center_point):
        for i, area in enumerate(self.parking_areas):
            if cv2.pointPolygonTest(area, center_point, False) >= 0:
                return i
        return -1
        
    def run_detection(self, show_video=True):
        if not self.load_model_and_areas():
            return False
            
        print("\nStarting detection...")
        
        cap = cv2.VideoCapture(self.video_path)
        if not cap.isOpened():
            print(f"Cannot open video: {self.video_path}")
            return False
            
        # Video properties
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        # Tracking
        parking_timers = defaultdict(lambda: None)
        parked_vehicles = set()
        vehicle_classes = [2, 3, 5, 7]  # car, motorcycle, bus, truck
        parking_threshold = 30  # seconds
        
        # Flickering/tracking issue detection
        last_seen = {}  # track_id -> frame_count when last seen
        lost_vehicles = {}  # track_id -> frame_count when lost
        tracking_gaps = defaultdict(list)  # track_id -> list of (lost_frame, found_frame) gaps
        
        # Stats
        frame_count = 0
        total_detections = 0
        start_time = time.time()
        fps_times = []
        
        print(f"Video: {total_frames} frames, {fps:.1f} FPS")
        if show_video:
            print("Press 'q' to stop early, 's' to toggle video display")
        else:
            print("Processing without video display (faster)")
        print()
        
        display_video = show_video
        
        while True:
            frame_start = time.time()
            
            ret, frame = cap.read()
            if not ret:
                break
                
            frame_count += 1
            current_time = frame_count / fps
            
            # Run detection
            results = self.model.track(frame, persist=True, classes=vehicle_classes, verbose=False)
            
            # Draw parking areas on frame
            if display_video:
                display_frame = frame.copy()
                for i, area in enumerate(self.parking_areas):
                    cv2.polylines(display_frame, [area], True, (0, 255, 0), 2)
                    # Add area label
                    center = area.mean(axis=0).astype(int)
                    cv2.putText(display_frame, f"Area {i+1}", tuple(center), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            # Process detections
            current_frame_vehicles = set()
            if results[0].boxes is not None and results[0].boxes.id is not None:
                boxes = results[0].boxes.xywh.cpu()
                track_ids = results[0].boxes.id.int().cpu().tolist()
                confidences = results[0].boxes.conf.float().cpu().tolist()
                
                for box, track_id, conf in zip(boxes, track_ids, confidences):
                    if conf < 0.5:
                        continue
                        
                    total_detections += 1
                    current_frame_vehicles.add(track_id)
                    
                    # Track flickering: check if this vehicle was lost and now found
                    if track_id in lost_vehicles:
                        lost_frame = lost_vehicles[track_id]
                        gap_duration = frame_count - lost_frame
                        tracking_gaps[track_id].append({
                            'lost_frame': lost_frame,
                            'found_frame': frame_count,
                            'gap_duration': gap_duration,
                            'gap_time_seconds': gap_duration / fps
                        })
                        print(f"Vehicle {track_id} re-tracked after {gap_duration} frames ({gap_duration/fps:.1f}s)")
                        del lost_vehicles[track_id]
                    
                    # Update last seen
                    last_seen[track_id] = frame_count
                    
                    # Get vehicle center and bounding box
                    x, y, w, h = box
                    center = (int(x), int(y))
                    
                    # Draw detection on frame
                    if display_video:
                        x1, y1 = int(x - w/2), int(y - h/2)
                        x2, y2 = int(x + w/2), int(y + h/2)
                        
                        # Check if in parking area
                        area_id = self.is_in_parking_area(center)
                        
                        if area_id >= 0:  # In parking area
                            # Check if vehicle is already marked as parked
                            if track_id in parked_vehicles:
                                color = (0, 0, 255)  # Red for parked vehicles
                                label = f"ID:{track_id} PARKED Area:{area_id+1}"
                            else:
                                color = (0, 255, 255)  # Yellow for vehicles in parking area but not yet parked
                                label = f"ID:{track_id} Area:{area_id+1}"
                        else:
                            color = (255, 0, 0)  # Blue for other vehicles
                            label = f"ID:{track_id}"
                            
                        cv2.rectangle(display_frame, (x1, y1), (x2, y2), color, 2)
                        cv2.putText(display_frame, label, (x1, y1-10), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                    
                    # Check if in parking area
                    area_id = self.is_in_parking_area(center)
                    
                    if area_id >= 0:  # In parking area
                        if track_id not in parking_timers:
                            parking_timers[track_id] = current_time
                            
                        # Check if parked long enough
                        parked_duration = current_time - parking_timers[track_id]
                        
                        if parked_duration >= parking_threshold and track_id not in parked_vehicles:
                            parked_vehicles.add(track_id)
                            
                            # Save parked vehicle data
                            self.results['parked_vehicles'].append({
                                'track_id': track_id,
                                'area_id': area_id + 1,
                                'time': current_time,
                                'frame': frame_count,
                                'duration': parked_duration
                            })
                            print(f"Vehicle {track_id} parked in area {area_id+1} after {parked_duration:.1f}s")
                    else:
                        # Vehicle left parking area
                        if track_id in parking_timers:
                            del parking_timers[track_id]
                        if track_id in parked_vehicles:
                            parked_vehicles.remove(track_id)
            
            # Detect lost vehicles (were seen recently but not in current frame)
            for vehicle_id, last_frame in list(last_seen.items()):
                if vehicle_id not in current_frame_vehicles and (frame_count - last_frame) == 1:
                    # Vehicle just lost tracking
                    lost_vehicles[vehicle_id] = frame_count
                    print(f"Vehicle {vehicle_id} lost tracking at frame {frame_count}")
            
            # Display video frame
            if display_video:
                # Add info overlay
                info_text = [
                    f"Frame: {frame_count}/{total_frames}",
                    f"FPS: {processing_fps:.1f}" if 'processing_fps' in locals() else "FPS: calculating...",
                    f"Detections: {total_detections}",
                    f"Parked: {len(parked_vehicles)}"
                ]
                
                for i, text in enumerate(info_text):
                    cv2.putText(display_frame, text, (10, 30 + i*25), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                
                cv2.imshow('Parking Detection', display_frame)
            
            # Calculate FPS
            frame_time = time.time() - frame_start
            processing_fps = 1.0 / frame_time if frame_time > 0 else 0
            fps_times.append(processing_fps)
            
            # Show progress
            if frame_count % 30 == 0:
                progress = (frame_count / total_frames) * 100
                avg_fps = sum(fps_times[-30:]) / min(30, len(fps_times))
                print(f"Progress: {progress:.1f}% | FPS: {avg_fps:.1f} | Parked: {len(parked_vehicles)}")
            
            # Check for quit or display toggle
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                print("Stopped by user")
                break
            elif key == ord('s'):
                display_video = not display_video
                if display_video:
                    print("Video display enabled")
                else:
                    print("Video display disabled")
                    cv2.destroyAllWindows()
                
        cap.release()
        cv2.destroyAllWindows()
        
        # Calculate final stats
        total_time = time.time() - start_time
        avg_fps = sum(fps_times) / len(fps_times) if fps_times else 0
        
        # Process tracking issues data
        total_gaps = sum(len(gaps) for gaps in tracking_gaps.values())
        vehicles_with_gaps = len(tracking_gaps)
        
        # Convert tracking gaps to list format for JSON serialization
        tracking_issues_list = []
        for vehicle_id, gaps in tracking_gaps.items():
            for gap in gaps:
                tracking_issues_list.append({
                    'track_id': vehicle_id,
                    'lost_frame': gap['lost_frame'],
                    'found_frame': gap['found_frame'],
                    'gap_duration_frames': gap['gap_duration'],
                    'gap_duration_seconds': gap['gap_time_seconds']
                })
        
        self.results['fps_data'] = fps_times
        self.results['tracking_issues'] = tracking_issues_list
        self.results['detection_stats'] = {
            'total_frames': frame_count,
            'total_detections': total_detections,
            'unique_parked': len(parked_vehicles),
            'processing_time': total_time,
            'tracking_gaps_total': total_gaps,
            'vehicles_with_tracking_issues': vehicles_with_gaps
        }
        self.results['performance'] = {
            'avg_fps': avg_fps,
            'min_fps': min(fps_times) if fps_times else 0,
            'max_fps': max(fps_times) if fps_times else 0
        }
        
        print(f"\nDetection completed!")
        print(f"Processed {frame_count} frames in {total_time:.1f}s")
        print(f"Average FPS: {avg_fps:.1f}")
        print(f"Parked vehicles detected: {len(parked_vehicles)}")
        print(f"Tracking issues: {total_gaps} gaps in {vehicles_with_gaps} vehicles")
        
        return True
        
    def save_results(self):
        # Create output directory
        output_dir = f"../output/experiments/{self.results['experiment_id']}"
        os.makedirs(output_dir, exist_ok=True)
        
        # Save JSON results
        json_file = f"{output_dir}/results.json"
        with open(json_file, 'w') as f:
            json.dump(self.results, f, indent=2, default=str)
            
        # Save CSV summary
        csv_file = f"{output_dir}/summary.csv"
        summary_data = {
            'experiment_id': [self.results['experiment_id']],
            'model': [self.model_name],
            'video': [self.video_name],
            'avg_fps': [self.results['performance']['avg_fps']],
            'min_fps': [self.results['performance']['min_fps']],
            'max_fps': [self.results['performance']['max_fps']],
            'total_frames': [self.results['detection_stats']['total_frames']],
            'total_detections': [self.results['detection_stats']['total_detections']],
            'parked_vehicles': [self.results['detection_stats']['unique_parked']],
            'processing_time': [self.results['detection_stats']['processing_time']],
            'tracking_gaps': [self.results['detection_stats']['tracking_gaps_total']],
            'vehicles_with_issues': [self.results['detection_stats']['vehicles_with_tracking_issues']]
        }
        
        df = pd.DataFrame(summary_data)
        df.to_csv(csv_file, index=False)
        
        # Save parked vehicles details
        if self.results['parked_vehicles']:
            parked_df = pd.DataFrame(self.results['parked_vehicles'])
            parked_df.to_csv(f"{output_dir}/parked_vehicles.csv", index=False)
            
        # Save tracking issues details
        if self.results['tracking_issues']:
            tracking_df = pd.DataFrame(self.results['tracking_issues'])
            tracking_df.to_csv(f"{output_dir}/tracking_issues.csv", index=False)
            
        print(f"\nResults saved to: {output_dir}")
        files_list = "results.json, summary.csv"
        if self.results['parked_vehicles']:
            files_list += ", parked_vehicles.csv"
        if self.results['tracking_issues']:
            files_list += ", tracking_issues.csv"
        print(f"Files: {files_list}")
        
        return output_dir

In [3]:
class ModelComparison:
    def __init__(self, video_path):
        self.video_path = video_path
        self.video_name = os.path.splitext(os.path.basename(video_path))[0]
        
        # Define all YOLOv8 models to test
        self.models = [
            'yolov8n.pt',  # Nano
            'yolov8s.pt',  # Small  
            'yolov8m.pt',  # Medium
            'yolov8l.pt',  # Large
            'yolov8x.pt'   # Extra Large
        ]
        
        self.model_dir = '../yolo-model'
        self.results = []
        self.comparison_id = f"comparison_{self.video_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        
        print(f"Model Comparison Setup:")
        print(f"Video: {self.video_path}")
        print(f"Models to test: {len(self.models)}")
        print(f"Comparison ID: {self.comparison_id}")
        
    def download_missing_models(self):
        """Download any missing YOLO models"""
        print("\nChecking for missing models...")
        
        for model_name in self.models:
            model_path = os.path.join(self.model_dir, model_name)
            
            if not os.path.exists(model_path):
                print(f"Downloading {model_name}...")
                try:
                    # Create model directory if it doesn't exist
                    os.makedirs(self.model_dir, exist_ok=True)
                    
                    # Download model (YOLO will handle this automatically)
                    temp_model = YOLO(model_name)
                    
                    # Move to our model directory
                    import shutil
                    source_path = os.path.join(os.getcwd(), model_name)
                    if os.path.exists(source_path):
                        shutil.move(source_path, model_path)
                        print(f"Downloaded and moved {model_name} to {model_path}")
                    else:
                        print(f"Model {model_name} downloaded to default location")
                        
                except Exception as e:
                    print(f"Failed to download {model_name}: {e}")
            else:
                print(f"Found {model_name}")
                
    def run_single_experiment(self, model_name, show_video=False):
        """Run experiment with a single model"""
        model_path = os.path.join(self.model_dir, model_name)
        
        print(f"\n{'='*50}")
        print(f"Testing Model: {model_name}")
        print(f"{'='*50}")
        
        try:
            detector = ParkingDetector(model_path, self.video_path)
            
            start_time = time.time()
            success = detector.run_detection(show_video=show_video)
            total_experiment_time = time.time() - start_time
            
            if success:
                # Save individual results
                output_dir = detector.save_results()
                
                # Extract key metrics for comparison
                stats = detector.results['detection_stats']
                perf = detector.results['performance']
                
                result = {
                    'model_name': model_name.replace('.pt', ''),
                    'model_path': model_path,
                    'experiment_id': detector.results['experiment_id'],
                    'output_dir': output_dir,
                    'total_experiment_time': total_experiment_time,
                    
                    # Performance metrics
                    'avg_fps': perf['avg_fps'],
                    'min_fps': perf['min_fps'],
                    'max_fps': perf['max_fps'],
                    
                    # Detection stats
                    'total_frames': stats['total_frames'],
                    'total_detections': stats['total_detections'],
                    'parked_vehicles': stats['unique_parked'],
                    'processing_time': stats['processing_time'],
                    
                    # Tracking quality
                    'tracking_gaps': stats.get('tracking_gaps_total', 0),
                    'vehicles_with_issues': stats.get('vehicles_with_tracking_issues', 0),
                    'tracking_quality_score': self.calculate_tracking_quality(stats),
                    
                    # Overall score
                    'overall_score': self.calculate_overall_score(perf, stats),
                    'success': True
                }
                
                print(f"SUCCESS: {model_name}")
                print(f"  Avg FPS: {perf['avg_fps']:.1f}")
                print(f"  Parked Vehicles: {stats['unique_parked']}")
                print(f"  Tracking Issues: {stats.get('tracking_gaps_total', 0)}")
                
            else:
                result = {
                    'model_name': model_name.replace('.pt', ''),
                    'model_path': model_path,
                    'success': False,
                    'error': 'Detection failed'
                }
                print(f"FAILED: {model_name}")
                
        except Exception as e:
            result = {
                'model_name': model_name.replace('.pt', ''),
                'model_path': model_path,
                'success': False,
                'error': str(e)
            }
            print(f"ERROR: {model_name} - {e}")
            
        return result
        
    def calculate_tracking_quality(self, stats):
        """Calculate tracking quality score (0-100)"""
        total_detections = stats.get('total_detections', 1)
        tracking_gaps = stats.get('tracking_gaps_total', 0)
        
        # Higher score = fewer tracking issues
        if total_detections == 0:
            return 0
            
        gap_ratio = tracking_gaps / total_detections
        quality_score = max(0, 100 - (gap_ratio * 100))
        return round(quality_score, 2)
        
    def calculate_overall_score(self, perf, stats):
        """Calculate overall model performance score (0-100)"""
        # Weighted scoring: FPS (30%), Tracking Quality (40%), Detection Count (30%)
        
        fps_score = min(100, (perf['avg_fps'] / 30) * 100)  # Normalize to 30 FPS max
        tracking_score = self.calculate_tracking_quality(stats)
        
        # Detection efficiency (detections per frame)
        detection_efficiency = stats.get('total_detections', 0) / max(1, stats.get('total_frames', 1))
        detection_score = min(100, detection_efficiency * 500)  # Scale appropriately
        
        overall = (fps_score * 0.3) + (tracking_score * 0.4) + (detection_score * 0.3)
        return round(overall, 2)
        
    def run_all_experiments(self, show_video=False):
        """Run experiments with all models"""
        print(f"Starting comparison with {len(self.models)} models...")
        print(f"Video display: {'Enabled' if show_video else 'Disabled (faster)'}")
        
        # Download missing models first
        self.download_missing_models()
        
        # Run experiments
        start_time = time.time()
        
        for i, model_name in enumerate(self.models, 1):
            print(f"\n[{i}/{len(self.models)}] Testing {model_name}...")
            
            result = self.run_single_experiment(model_name, show_video)
            self.results.append(result)
            
        total_time = time.time() - start_time
        
        print(f"\n{'='*60}")
        print(f"ALL EXPERIMENTS COMPLETED in {total_time:.1f}s")
        print(f"{'='*60}")
        
        return self.results
        
    def save_comparison_results(self):
        """Save consolidated comparison results"""
        output_dir = f"../output/comparisons/{self.comparison_id}"
        os.makedirs(output_dir, exist_ok=True)
        
        # Save detailed JSON results
        json_file = f"{output_dir}/comparison_results.json"
        with open(json_file, 'w') as f:
            json.dump({
                'comparison_id': self.comparison_id,
                'video_path': self.video_path,
                'video_name': self.video_name,
                'timestamp': datetime.now().isoformat(),
                'models_tested': len(self.models),
                'results': self.results
            }, f, indent=2, default=str)
            
        # Create comparison CSV
        successful_results = [r for r in self.results if r.get('success', False)]
        
        if successful_results:
            df = pd.DataFrame(successful_results)
            
            # Select key columns for comparison
            comparison_cols = [
                'model_name', 'avg_fps', 'total_detections', 'parked_vehicles',
                'tracking_gaps', 'vehicles_with_issues', 'tracking_quality_score',
                'overall_score', 'processing_time', 'total_experiment_time'
            ]
            
            comparison_df = df[comparison_cols].copy()
            comparison_df = comparison_df.sort_values('overall_score', ascending=False)
            
            csv_file = f"{output_dir}/model_comparison.csv"
            comparison_df.to_csv(csv_file, index=False)
            
            print(f"\nComparison results saved:")
            print(f"  Directory: {output_dir}")
            print(f"  Files: comparison_results.json, model_comparison.csv")
            
            return output_dir, comparison_df
        else:
            print(f"No successful experiments to save comparison")
            return output_dir, None
            
    def display_results(self):
        """Display comparison results in a nice format"""
        successful_results = [r for r in self.results if r.get('success', False)]
        
        if not successful_results:
            print("No successful experiments to display")
            return
            
        print(f"\nMODEL COMPARISON RESULTS")
        print(f"{'='*80}")
        print(f"Video: {self.video_name}")
        print(f"Models tested: {len(successful_results)}/{len(self.models)}")
        print()
        
        # Sort by overall score
        sorted_results = sorted(successful_results, key=lambda x: x['overall_score'], reverse=True)
        
        # Display ranking
        print(f"{'Rank':<4} {'Model':<8} {'Score':<6} {'FPS':<6} {'Parked':<7} {'Gaps':<5} {'Quality':<8}")
        print(f"{'-'*50}")
        
        for i, result in enumerate(sorted_results, 1):
            print(f"{i:<4} {result['model_name']:<8} {result['overall_score']:<6} "
                  f"{result['avg_fps']:<6.1f} {result['parked_vehicles']:<7} "
                  f"{result['tracking_gaps']:<5} {result['tracking_quality_score']:<8.1f}")
                  
        # Show detailed stats for top performer
        best_model = sorted_results[0]
        print(f"\nBEST PERFORMER: {best_model['model_name']}")
        print(f"  Overall Score: {best_model['overall_score']}")
        print(f"  Average FPS: {best_model['avg_fps']:.1f}")
        print(f"  Total Detections: {best_model['total_detections']}")
        print(f"  Parked Vehicles: {best_model['parked_vehicles']}")
        print(f"  Tracking Quality: {best_model['tracking_quality_score']:.1f}")
        print(f"  Processing Time: {best_model['processing_time']:.1f}s")
        
        return sorted_results

In [4]:
# COMPARISON CONFIGURATION

# Video path for testing
video_path = '../dataset/video_1_cut_01-00_01-10.mov'

# Create comparison instance
comparison = ModelComparison(video_path)

print("Ready to run model comparison!")
print("\nTo start the comparison, run the next cell.")
print("Note: This will take significant time as it tests all 5 models.")

Model Comparison Setup:
Video: ../dataset/video_1_cut_01-00_01-10.mov
Models to test: 5
Comparison ID: comparison_video_1_cut_01-00_01-10_20251002_000229
Ready to run model comparison!

To start the comparison, run the next cell.
Note: This will take significant time as it tests all 5 models.


In [5]:
# RUN MODEL COMPARISON

print("Starting automated model comparison...")
print("This will test all YOLOv8 models from nano to extra-large")
print()

# Run all experiments (set show_video=True if you want to see each one)
results = comparison.run_all_experiments(show_video=False)

# Save consolidated results
output_dir, comparison_df = comparison.save_comparison_results()

# Display results summary
ranking = comparison.display_results()

Starting automated model comparison...
This will test all YOLOv8 models from nano to extra-large

Starting comparison with 5 models...
Video display: Disabled (faster)

Checking for missing models...
Found yolov8n.pt
Found yolov8s.pt
Found yolov8m.pt
Found yolov8l.pt
Found yolov8x.pt

[1/5] Testing yolov8n.pt...

Testing Model: yolov8n.pt
Model: yolov8n
Video: video_1_cut_01-00_01-10
Looking for areas: ../output/parking-areas-video_1_cut_01-00_01-10.json

Loading model...
Model loaded: yolov8n
Loaded 2 parking areas

Starting detection...
Video: 300 frames, 30.0 FPS
Processing without video display (faster)

Vehicle 7 lost tracking at frame 3
Vehicle 10 lost tracking at frame 3
Vehicle 7 lost tracking at frame 3
Vehicle 10 lost tracking at frame 3
Vehicle 10 re-tracked after 4 frames (0.1s)
Vehicle 10 lost tracking at frame 9
Vehicle 6 lost tracking at frame 10
Vehicle 10 re-tracked after 4 frames (0.1s)
Vehicle 10 lost tracking at frame 9
Vehicle 6 lost tracking at frame 10
Vehicle 6 

In [6]:
# DETAILED RESULTS ANALYSIS

if 'comparison_df' in locals() and comparison_df is not None:
    print("\nDETAILED ANALYSIS")
    print("="*60)
    
    # Performance analysis
    print("\nPERFORMANCE METRICS:")
    print(f"Fastest Model: {comparison_df.loc[comparison_df['avg_fps'].idxmax(), 'model_name']} "
          f"({comparison_df['avg_fps'].max():.1f} FPS)")
    print(f"Most Accurate: {comparison_df.loc[comparison_df['total_detections'].idxmax(), 'model_name']} "
          f"({comparison_df['total_detections'].max()} detections)")
    print(f"Best Tracking: {comparison_df.loc[comparison_df['tracking_quality_score'].idxmax(), 'model_name']} "
          f"({comparison_df['tracking_quality_score'].max():.1f} quality score)")
    
    # Speed vs Accuracy trade-off
    print(f"\nSPEED vs ACCURACY TRADE-OFF:")
    for _, row in comparison_df.iterrows():
        speed_category = "Fast" if row['avg_fps'] > 15 else "Medium" if row['avg_fps'] > 10 else "Slow"
        accuracy_category = "High" if row['total_detections'] > comparison_df['total_detections'].mean() else "Low"
        print(f"  {row['model_name']}: {speed_category} speed, {accuracy_category} accuracy")
    
    # Tracking issues analysis
    print(f"\nTRACKING ISSUES ANALYSIS:")
    total_gaps = comparison_df['tracking_gaps'].sum()
    total_vehicles_with_issues = comparison_df['vehicles_with_issues'].sum()
    
    if total_gaps > 0:
        print(f"Total tracking gaps across all models: {total_gaps}")
        print(f"Models with fewest tracking issues:")
        sorted_by_gaps = comparison_df.sort_values('tracking_gaps')
        for _, row in sorted_by_gaps.head(3).iterrows():
            print(f"  {row['model_name']}: {row['tracking_gaps']} gaps")
    else:
        print("No tracking issues detected in any model!")
    
    # Recommendations
    print(f"\nRECOMMENDations:")
    best_overall = comparison_df.loc[comparison_df['overall_score'].idxmax()]
    fastest = comparison_df.loc[comparison_df['avg_fps'].idxmax()]
    most_accurate = comparison_df.loc[comparison_df['total_detections'].idxmax()]
    
    print(f"  Best Overall: {best_overall['model_name']} (score: {best_overall['overall_score']})")
    print(f"  For Real-time: {fastest['model_name']} ({fastest['avg_fps']:.1f} FPS)")
    print(f"  For Accuracy: {most_accurate['model_name']} ({most_accurate['total_detections']} detections)")
    
else:
    print("No successful experiments to analyze")
    print("Make sure to run the comparison first!")


DETAILED ANALYSIS

PERFORMANCE METRICS:
Fastest Model: yolov8n (13.2 FPS)
Most Accurate: yolov8l (2750 detections)
Best Tracking: yolov8l (98.4 quality score)

SPEED vs ACCURACY TRADE-OFF:
  yolov8n: Medium speed, Low accuracy
  yolov8s: Slow speed, Low accuracy
  yolov8m: Slow speed, High accuracy
  yolov8l: Slow speed, High accuracy
  yolov8x: Slow speed, High accuracy

TRACKING ISSUES ANALYSIS:
Total tracking gaps across all models: 278
Models with fewest tracking issues:
  yolov8l: 45 gaps
  yolov8m: 47 gaps
  yolov8x: 53 gaps

RECOMMENDations:
  Best Overall: yolov8n (score: 81.92)
  For Real-time: yolov8n (13.2 FPS)
  For Accuracy: yolov8l (2750 detections)


In [7]:
# EXPORT RESULTS FOR FURTHER ANALYSIS

if 'comparison' in locals() and comparison.results:
    print("EXPORT OPTIONS:")
    print("="*40)
    
    # Show available data
    successful_count = len([r for r in comparison.results if r.get('success', False)])
    print(f"Successfully tested: {successful_count}/{len(comparison.models)} models")
    
    if successful_count > 0:
        print(f"\nResults saved to: {output_dir}")
        print(f"Files available:")
        print(f"  - comparison_results.json (detailed data)")
        print(f"  - model_comparison.csv (summary table)")
        
        # Show individual experiment directories
        print(f"\nIndividual experiment results:")
        for result in comparison.results:
            if result.get('success', False):
                print(f"  {result['model_name']}: {result.get('output_dir', 'N/A')}")
                
        print(f"\nYou can now:")
        print(f"  1. Open CSV files in Excel/Google Sheets")
        print(f"  2. Use JSON data for custom analysis") 
        print(f"  3. Compare individual experiment details")
        print(f"  4. Create custom visualizations")
        
        # Quick stats for copying
        print(f"\nQUICK STATS (copy-friendly):")
        if 'comparison_df' in locals() and comparison_df is not None:
            print("Model,FPS,Detections,Parked,TrackingGaps,OverallScore")
            for _, row in comparison_df.iterrows():
                print(f"{row['model_name']},{row['avg_fps']:.1f},{row['total_detections']},"
                      f"{row['parked_vehicles']},{row['tracking_gaps']},{row['overall_score']}")
    else:
        print("No successful experiments to export")
        
else:
    print("No comparison data available")
    print("Run the model comparison first!")

EXPORT OPTIONS:
Successfully tested: 5/5 models

Results saved to: ../output/comparisons/comparison_video_1_cut_01-00_01-10_20251002_000229
Files available:
  - comparison_results.json (detailed data)
  - model_comparison.csv (summary table)

Individual experiment results:
  yolov8n: ../output/experiments/yolov8n_video_1_cut_01-00_01-10_20251002_000229
  yolov8s: ../output/experiments/yolov8s_video_1_cut_01-00_01-10_20251002_000257
  yolov8m: ../output/experiments/yolov8m_video_1_cut_01-00_01-10_20251002_000349
  yolov8l: ../output/experiments/yolov8l_video_1_cut_01-00_01-10_20251002_000523
  yolov8x: ../output/experiments/yolov8x_video_1_cut_01-00_01-10_20251002_000727

You can now:
  1. Open CSV files in Excel/Google Sheets
  2. Use JSON data for custom analysis
  3. Compare individual experiment details
  4. Create custom visualizations

QUICK STATS (copy-friendly):
Model,FPS,Detections,Parked,TrackingGaps,OverallScore
yolov8n,13.2,1813,0,58,81.92
yolov8s,6.4,2209,0,75,75.04
yolov8m